In [1]:
import torch
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
posts = pd.read_csv('r_udub_posts.csv')

In [3]:
posts.head(5)

,title,selftext,link_flair_text,id,url,num_comments,score
0,Any UW redditors want to meet up Thursday 10/2...,"We failed on 10/22, but I think with a week of...",NaN,9y4hg,https://www.reddit.com/r/udub/comments/9y4hg/a...,6,4
1,We need a UW-ified logo.,If someone here has arcane skill in the graphi...,NaN,9ywtc,https://www.reddit.com/r/udub/comments/9ywtc/w...,2,3
2,Thursday bowling success!,[deleted],NaN,9z66c,https://www.reddit.com/r/udub/comments/9z66c/t...,2,3
3,"Next UW meetup: Thursday, 11/5 at 11:00am in t...",This time we will be playing ping pong followe...,NaN,a0ail,https://www.reddit.com/r/udub/comments/a0ail/n...,2,4
4,Next meetup: December 3rd. Need ideas,"Alright, so who is up for a December 3rd meetu...",NaN,a9lq8,https://www.reddit.com/r/udub/comments/a9lq8/n...,7,2


In [9]:
posts['combined_text'] = posts['title'] + " " + posts['selftext'].fillna("")
flair_categories = ["admissions", "academics", "student life", "advice", "discussion", "meme", "rant", "psa", "event", "poll"]

flairedNotSelf = posts[(posts['link_flair_text'].notnull()) & (posts['selftext'] != '[removed]') & (posts['selftext'] != '[deleted]') & posts['selftext'].notnull()]
ModelDataLower = flairedNotSelf.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)
ModelDataFiltered = ModelDataLower[ModelDataLower['link_flair_text'].isin(flair_categories)][['combined_text', 'link_flair_text']]

ModelDataFiltered.head()

,combined_text,link_flair_text
28066,thoughts on madrona? i have an emotional suppo...,discussion
28077,soc 222 anyone has took or taking soc222(socio...,academics
28080,betsy evans - ling/anth 233 does anyone have a...,academics
28083,tell me what you want from remote teaching? th...,discussion
28088,efs experience/thoughts/opinions! i just regis...,discussion


In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the input data
encoded_body = tokenizer(ModelDataFiltered['combined_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Encode the labels
label_encoder = LabelEncoder()
encoded_flairs = label_encoder.fit_transform(ModelDataFiltered['link_flair_text'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(encoded_body['input_ids'], encoded_flairs, test_size=0.2, random_state=52)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

7224
1807
7224
1807


In [6]:
train_dataset = TensorDataset(X_train, torch.tensor(y_train))
test_dataset = TensorDataset(X_test, torch.tensor(y_test))
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(set(encoded_flairs)))

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [8]:
from tqdm import tqdm

model.train()
for epoch in range(4):
    print(f'Epoch {epoch}')
    for i, batch in enumerate(tqdm(train_loader)):
        batch = [item.to(device) for item in batch]
        inputs, labels = batch
        inputs = inputs.long()
        labels = labels.long()
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"end of Epoch {epoch}")


Epoch 0


100%|██████████| 362/362 [11:13<00:00,  1.86s/it]


end of Epoch 0
Epoch 1


100%|██████████| 362/362 [11:12<00:00,  1.86s/it]


end of Epoch 1
Epoch 2


100%|██████████| 362/362 [11:11<00:00,  1.86s/it]


end of Epoch 2
Epoch 3


100%|██████████| 362/362 [11:11<00:00,  1.86s/it]

end of Epoch 3


In [10]:
model.eval()
predictions = []
true_labels = []
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [11]:
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        batch = [item.to(device) for item in batch]
        inputs, labels = batch
        outputs = model(inputs)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

100%|██████████| 226/226 [00:52<00:00,  4.27it/s]


In [12]:
accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.5633646928610957
